# Sketchmap Demo

In [ ]:
import os
from os import path
import re
import time
import threading
import subprocess

import ipywidgets as widgets
from IPython.display import display

In [ ]:
log_fn = "sketchmap.log"
process = None

def output_worker():
    pre_tag = '<pre style="width:600px; max-height:250px; overflow-x:auto; line-height:1em; font-size:0.8em;">'   
    dots = 0
    latest = ""
    while(process.poll() == None):
        time.sleep(1)
        # update output window
        if os.path.exists(log_fn):
            full = open(log_fn).read() # TODO seek forward
            full = re.sub("\n.*\r", "\n", full.strip("\r")) # handle carriage return
            output = "\n".join(full.split("\n")[-20:]) #last 100 lines
        else:
            output = ""
        dots += 1
        if(latest!=output): # new output
            dots = 0 # rest dot counter
        latest = output
        output += "\n" + ("."*dots) + "\n"
        output_area.value = pre_tag + output + '</pre>'
    
    # read one last time entirely
    output = open(log_fn).read()
    output = re.sub("\n.*\r", "\n", output.strip("\r")) # handle carriage return
    output += "\n\nProcess finished, exit code: %s"%process.returncode
    output_area.value = pre_tag + output + '</pre>'
    btn_startstop.description="Start"
    
def start_process():
    global process
    btn_startstop.description="Stop"
    logfile = open(log_fn, "w")
    cmd = ["./create_new_sketchmap.sh", "traj.lys.dipeptide.bare"]
    process = subprocess.Popen(cmd, stdout=logfile, stderr=logfile)
    threading.Thread(target=output_worker).start() 
    
def on_click_startstop(e):
    if(btn_startstop.description.startswith("Start")):
        start_process()
    else:
        process.kill()
     
btn_startstop = widgets.Button(description="Start")
btn_startstop.on_click(on_click_startstop)
output_area = widgets.HTML()
display(widgets.VBox([btn_startstop, output_area]))